# Spatial Data Processing with PySAL & Pandas


In [4]:
#by convention, we use these shorter two-letter names
import pysal as ps
import pandas as pd
import numpy as np

PySAL has simple way to read in data. But, first, you need to get the path from where your notebook is running on your computer to the place the data is. For example, to find where the notebook is running:

In this we process the pakistan district shapefile having attribute data of population of 1981 and 1998, also some more infromation i.e. Male Female ratio, population density etc.

In [9]:
cd C:\Users\HAMMA\Desktop\gdp project\2012-2015

C:\Users\HAMMA\Desktop\gdp project\2012-2015


### Working with shapefiles

We can work with local files, to read in a shapefile, we will need the path to the file.

In [42]:
shp_path = r"C:\Users\HAMMA\Desktop\gdp project\2012-2015\pak_dist4.shp"
print(shp_path)

C:\Users\HAMMA\Desktop\gdp project\2012-2015\pak_dist4.shp


Then, we open the file using the `ps.open` command:

In [43]:
f = ps.open(shp_path)

`f` is what we call a "file handle." That means that it only *points* to the data and provides ways to work with it. By itself, it does not read the whole dataset into memory. To see basic information about the file, we can use a few different methods. 

For instance, the header of the file, which contains most of the metadata about the file:

In [44]:
f.header

{'BBOX Mmax': 0.0,
 'BBOX Mmin': 0.0,
 'BBOX Xmax': 75.36699676513683,
 'BBOX Xmin': 60.89943695068354,
 'BBOX Ymax': 36.889804840088004,
 'BBOX Ymin': 23.702915191650447,
 'BBOX Zmax': 0.0,
 'BBOX Zmin': 0.0,
 'File Code': 9994,
 'File Length': 969710,
 'Shape Type': 5,
 'Unused0': 0,
 'Unused1': 0,
 'Unused2': 0,
 'Unused3': 0,
 'Unused4': 0,
 'Version': 1000}

To actually read in the shapes from memory, you can use the following commands:

In [45]:
f.by_row(14) #gets the 14th shape from the file

In [46]:
all_polygons = f.read() #reads in all polygons from memory

In [47]:
len(all_polygons)

107

So, all 107 polygons have been read in from file.
           
           "This is not the full pakistan shape file because it is the old data of population, which doen't have all district information of present time"

These are stored in PySAL shape objects, which can be used by PySAL and can be converted to other Python shape objects.
They typically have a few methods. So, since we've read in polygonal data, we can get some properties about the polygons. Let's just have a look at the first polygon:

In [48]:
all_polygons[0:5]

In [49]:
all_polygons[0].centroid #the centroid of the first polygon

(65.35974546861902, 26.265346073984496)

In [50]:
all_polygons[0].area

2.045215825215557

In [51]:
all_polygons[0].perimeter

8.063774483083392

While in the Jupyter Notebook, you can examine what properties an object has by using the tab key.

In [52]:
polygon = all_polygons[0]

In [53]:
polygon.area #press tab when the cursor is right after the dot

2.045215825215557

### Working with Data Tables

In [54]:
dbf_path = r"C:\Users\HAMMA\Desktop\gdp project\2012-2015\pak_dist4.dbf"
print(dbf_path)

C:\Users\HAMMA\Desktop\gdp project\2012-2015\pak_dist4.dbf


When you're working with tables of data, like a `csv` or `dbf`, you can extract your data in the following way. Let's open the dbf file we got the path for above.

In [55]:
f = ps.open(dbf_path)

Just like with the shapefile, we can examine the header of the dbf file.

In [56]:
f.header

[u'ISO',
 u'NAME_1',
 u'NAME_2',
 u'NAME_3',
 u'OBJECTID',
 u'Area',
 u'F1981__pop',
 u'F1998__pop',
 u'Male',
 u'Female',
 u'Sex_ratio',
 u'Pop_Sq_Km',
 u'Ave_H_Hol_',
 u'F1981_98_G',
 u'OID_',
 u'OBJECTID_1',
 u'DIST_NAME',
 u'Area_1',
 u'F1981__p_1',
 u'F1998__p_1',
 u'Male_1',
 u'Female_1',
 u'Sex_rati_1',
 u'Pop_Sq_K_1',
 u'Ave_H_Hol1',
 u'F1981_98_1']

So, the header is a list containing the names of all of the fields we can read.
If we just wanted to grab the data of interest, `F1998__pop`, we can use either `by_col` or `by_col_array`, depending on the format we want the resulting data in:

In [57]:
pop1998 = f.by_col('F1998__pop')
print(type(pop1998).__name__, pop1998[8:20])
pop1998 = f.by_col_array('F1998__pop')
print(type(pop1998).__name__, pop1998[8:20])

('list', [25648, 155488, 225028, 58868, 129412, 108736, 196330, 412064, 97316, 97332, 17304, 54365])
('ndarray', array([[ 25648],
       [155488],
       [225028],
       [ 58868],
       [129412],
       [108736],
       [196330],
       [412064],
       [ 97316],
       [ 97332],
       [ 17304],
       [ 54365]]))


As you can see, the `by_col` function returns a list of data, with no shape. It can only return one column at a time:

In [58]:
HRs = f.by_col('F1981__pop','F1998__pop')

TypeError: __call__() takes exactly 2 arguments (3 given)

This error message is called a "traceback," as you see in the top right, and it usually provides feedback on why the previous command did not execute correctly. Here, you see that one-too-many arguments was provided to `__call__`, which tells us we cannot pass as many arguments as we did to `by_col`.

If you want to read in many columns at once and store them to an array, use `by_col_array`:

In [59]:
HRs = f.by_col_array('F1981__pop','F1998__pop')

In [60]:
HRs[8:18]

array([[234051,  25648],
       [288056, 155488],
       [432817, 225028],
       [109941,  58868],
       [245894, 129412],
       [202564, 108736],
       [367183, 196330],
       [759941, 412064],
       [181310,  97316],
       [180398,  97332]])

It is best to use `by_col_array` on data of a single type. That is, if you read in a lot of columns, some of them numbers and some of them strings, all columns will get converted to the same datatype:

In [61]:
allcolumns = f.by_col_array(['NAME_2','NAME_3','F1981__pop','F1998__pop'])

In [62]:
allcolumns

array([[u'Kalat', u'Awaran', u'118173', u'62114'],
       [u'Kalat', u'Kalat', u'1457722', u'767137'],
       [u'Kalat', u'Kharan', u'206909', u'107261'],
       [u'Kalat', u'Khuzdar', u'417466', u'220023'],
       [u'Kalat', u'Lasbela', u'312695', u'167470'],
       [u'Kalat', u'Mastung', u'164645', u'87334'],
       [u'Makran', u'Gwadar', u'185498', u'99436'],
       [u'Makran', u'Kech', u'413204', u'216566'],
       [u'Makran', u'Panjgur', u'234051', u'25648'],
       [u'Nasirabad', u'Bolan', u'288056', u'155488'],
       [u'Nasirabad', u'Jafarabad', u'432817', u'225028'],
       [u'Nasirabad', u'Jhal Magsi', u'109941', u'58868'],
       [u'Nasirabad', u'Nasirabad', u'245894', u'129412'],
       [u'Quetta', u'Chagai', u'202564', u'108736'],
       [u'Quetta', u'Pishin', u'367183', u'196330'],
       [u'Quetta', u'Quetta', u'759941', u'412064'],
       [u'Sibi', u'Dera Bugti', u'181310', u'97316'],
       [u'Sibi', u'Sibi', u'180398', u'97332'],
       [u'Sibi', u'Ziarat', u'33340', 

Note that the numerical columns, `NAME_2` & `NAME_3` etc are now considered strings, since they show up with the single tickmarks around them, like `'0.0'`.

### Using Pandas with PySAL

A new functionality added to PySAL recently allows you to work with shapefile/dbf pairs using Pandas. This *optional* extension is only turned on if you have Pandas installed. The extension is the `ps.pdio` module:

In [64]:
ps.pdio

<module 'pysal.contrib.pdio' from 'C:\Users\HAMMA\Anaconda2\lib\site-packages\pysal\contrib\pdio\__init__.pyc'>

To use it, you can read in shapefile/dbf pairs using the `ps.pdio.read_files` command. 

In [65]:
#shp_path = ps.examples.get_path('NAT.shp')
data_table = ps.pdio.read_files(shp_path)

This reads in *the entire database table* and adds a column to the end, called `geometry`, that stores the geometries read in from the shapefile. 

Now, you can work with it like a standard pandas dataframe.

In [66]:
data_table.head()

,ISO,NAME_1,NAME_2,NAME_3,OBJECTID,Area,F1981__pop,F1998__pop,Male,Female,...,Area_1,F1981__p_1,F1998__p_1,Male_1,Female_1,Sex_rati_1,Pop_Sq_K_1,Ave_H_Hol1,F1981_98_1,geometry
0,PAK,Baluchistan,Kalat,Awaran,3,29510,118173,62114,56059,110353,...,29510,118173,62114,56059,110353,110.8,4.0,5.4,0.40,<pysal.cg.shapes.Polygon object at 0x000000000...
1,PAK,Baluchistan,Kalat,Kalat,36,140612,1457722,767137,690585,1044174,...,140612,1457722,767137,690585,1044174,111.1,10.4,6.0,1.98,<pysal.cg.shapes.Polygon object at 0x000000000...
2,PAK,Baluchistan,Kalat,Kharan,46,48051,206909,107261,99648,128040,...,48051,206909,107261,99648,128040,107.6,4.3,5.8,2.86,<pysal.cg.shapes.Polygon object at 0x000000000...
3,PAK,Baluchistan,Kalat,Khuzdar,48,35380,417466,220023,197443,276449,...,35380,417466,220023,197443,276449,111.4,11.8,5.4,2.45,<pysal.cg.shapes.Polygon object at 0x000000000...
4,PAK,Baluchistan,Kalat,Lasbela,57,15153,312695,167470,145225,188139,...,15153,312695,167470,145225,188139,115.3,20.6,6.4,3.03,<pysal.cg.shapes.Polygon object at 0x000000000...


The `read_files` function only works on shapefile/dbf pairs. If you need to read in data using CSVs, use pandas directly:

The nice thing about working with pandas dataframes is that they have very powerful baked-in support for relational-style queries. By this, I mean that it is very easy to find things like:

The number of counties in each state:

In [67]:
data_table.groupby('NAME_2').size()

NAME_2
Bahawalpur          3
Bannu               2
Dera Ghazi Khan     4
Dera Ismail Khan    2
F.A.T.A.            7
Faisalabad          3
Gujranwala          5
Hazara              5
Hyderabad           5
Kalat               6
Karachi             5
Kohat               3
Lahore              4
Larkana             3
Makran              3
Malakand            4
Mardan              3
Mirpur Khas         3
Multan              6
Nasirabad           4
Northern Areas      1
Peshawar            3
Quetta              3
Rawalpindi          4
Sargodha            4
Sibi                3
Sukkur              5
Zhob                4
dtype: int64

Or, to get the rows of the table that are in FATA, we can use the `query` function of the dataframe:

In [68]:
data_table.query('NAME_2 == "F.A.T.A."')

,ISO,NAME_1,NAME_2,NAME_3,OBJECTID,Area,F1981__pop,F1998__pop,Male,Female,...,Area_1,F1981__p_1,F1998__p_1,Male_1,Female_1,Sex_rati_1,Pop_Sq_K_1,Ave_H_Hol1,F1981_98_1,geometry
23,PAK,F.A.T.A.,F.A.T.A.,Bajaur,7,1290,595227,305137,289206,290090,...,1290,595227,305137,289206,290090,105.2,461.4,9.1,4.33,<pysal.cg.shapes.Polygon object at 0x000000000...
24,PAK,F.A.T.A.,F.A.T.A.,Khyber,49,2576,546730,284602,284256,262128,...,2576,546730,284602,284256,262128,108.6,212.2,10.0,3.92,<pysal.cg.shapes.Polygon object at 0x000000000...
25,PAK,F.A.T.A.,F.A.T.A.,Kurram,53,3380,448310,229634,294362,218676,...,3380,448310,229634,294362,218676,105.0,132.6,10.6,2.50,<pysal.cg.shapes.Polygon object at 0x000000000...
26,PAK,F.A.T.A.,F.A.T.A.,Mohmand,68,2296,334453,175404,163933,159049,...,2296,334453,175404,163933,159049,110.3,145.7,9.0,4.28,<pysal.cg.shapes.Polygon object at 0x000000000...
27,PAK,F.A.T.A.,F.A.T.A.,N. Waziristan,72,4707,361246,192432,238910,168814,...,4707,361246,192432,238910,168814,114.0,76.7,9.1,2.46,<pysal.cg.shapes.Polygon object at 0x000000000...
28,PAK,F.A.T.A.,F.A.T.A.,Orakzai,79,1538,225441,112766,358751,112675,...,1538,225441,112766,358751,112675,100.1,146.6,8.8,-2.69,<pysal.cg.shapes.Polygon object at 0x000000000...
29,PAK,F.A.T.A.,F.A.T.A.,S. Waziristan,88,6620,429841,231080,309454,198761,...,6620,429841,231080,309454,198761,116.3,64.9,8.5,1.95,<pysal.cg.shapes.Polygon object at 0x000000000...


Behind the scenes, this uses a fast vectorized library, `numexpr`, to essentially do the following. 

First, compare each row's `NAME_2` column to `'F.A.T.A.'` and return `True` if the row matches:

In [69]:
data_table.NAME_2 == 'F.A.T.A.'

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23      True
24      True
25      True
26      True
27      True
28      True
29      True
       ...  
77     False
78     False
79     False
80     False
81     False
82     False
83     False
84     False
85     False
86     False
87     False
88     False
89     False
90     False
91     False
92     False
93     False
94     False
95     False
96     False
97     False
98     False
99     False
100    False
101    False
102    False
103    False
104    False
105    False
106    False
Name: NAME_2, Length: 107, dtype: bool

Then, use that to filter out rows where the condition is true: